In [ ]:
import Helpful_functions as hf
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In this notebook, we are going to perform a data generation and fitting procedure for a rigid body with body velocity $v(t)$ and angular velocity $\omega(t)$. We then fit a DMD model on this using a Lie Series to see if we can reconstruct the dynamics.

First, we generate the data

In [ ]:
def step(p,R,v,omega,delta_t):
    # Method to step forward the dynamics my a small step
    R_step = sp.linalg.expm(delta_t*hf.hat(omega))
    return p+R@v*delta_t, R@R_step 
    
def v(t):
    # Velocity in the body frame
    return np.array([np.cos(t),3,np.sin(t)])

def omega(t):
    # Angular velocity in the body frame
    return np.array([np.cos(t),np.sin(t),np.cos(4*t)])

#Random initial conditions
p0 = np.zeros(3)
R0 = hf.sample_SO3()

#Time Parameters
tmin = 0
tmax = 10
n = 100
delta_t = (tmax-tmin)/n
#Initialize state
p = p0
R = R0
v0 = v(tmin)
omega0 = omega(tmin)
#Initialize arrays
times = np.linspace(tmin,tmax,n)
p_arr = np.zeros((3,len(times)))
R_arr = np.zeros((3,3,len(times)))
v_arr = np.zeros((3,len(times)))
omega_r = np.zeros((3,len(times)))
#Populate arrays
p_arr[:,0] = p0
R_arr[:,:,0] = R0
v_arr[:,0] = v0
omega_r[:,0] = omega0
for k in range(len(times)-1):
    p,R = step(p,R,v(times[k]),omega(times[k]),delta_t)
    p_arr[:,k+1] = p
    R_arr[:,:,k+1] = R
    v_arr[:,k+1] = v(times[k+1])
    omega_r[:,k+1] = omega(times[k+1])

Let's go ahead and plot the position of our particle

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
x = p_arr[0,:]
y = p_arr[1,:]
z = p_arr[2,:]
ax.plot(x, y, z,color="orange")
plt.show()

## Now let's perform the lifitng

In [ ]:
def form_lifting(p,v,R,omega,n,LCT=None,ang_max = 10):
    """
    Function to perform lifting of state into quaternionic frame

    Parameters:
    -----------
    p: Position in space
    v: Velocity in space
    R: Initial rotational configuration given by an element of SO(3)
    omega: Angular velocity about the moving frame
    """
    omega_hat = hf.hat(omega/ang_max,LCT=LCT)
    lifted = np.zeros(24+9*n)
    lifted[0:3] = p
    lifted[3:6] = v
    lifted[6:15] = R.flatten()
    start_index=24
    lifted[15:24] = omega_hat.flatten()
    interm = R
    for k in range(n):
        interm = R@omega_hat
        lifted[start_index+9*k:start_index+9*(k+1)] = interm.flatten()
    return lifted

def form_lifting_delay(p_arr,v_arr,R_arr,omega_arr,n,ang_max,LCT = None):
    """
    Used for lifting a delay embedded version for faster 
    initial condition recovery

    Parameters:
    ----------
    p_arr: np.array: Array of position vectors, columns are snapshots
    v_arr: np.array: Array of velocity vectors, columns are snapshots
    R_arr: np.array: Tensor of Rotation matrices, last index corresponds to each snapshot
    omega_arr: np.array: Array of angular velocity vectors, last index 
        corresponds to each snapshot

    Outputs:
    -------
    lifted_arr: np.array: Lifted array
    """
    m = p_arr.shape[-1]
    base_size = 24+9*n
    lifted_arr = np.zeros(m*base_size)
    if not np.all(LCT):
        LCT = hf.levi_cevita_tensor()
    for k in range(m):
        curr_p = p_arr[:,k]
        curr_v = v_arr[:,k]
        curr_R = R_arr[:,:,k]
        curr_omega = omega_arr[:,k]
        lifted_arr[k*base_size:(k+1)*base_size] = form_lifting(p=curr_p,v=curr_v,R=curr_R,
                                                               omega=curr_omega,n=n,LCT=LCT,ang_max=ang_max)
    return lifted_arr


def basic_DMD(X,Y,rrr=False):
    """
    Basic Exact Dynamic Mode Decomposition with
    the option to use Rayleigh-Ritz Refinement (RRR)

    Parameters:
    ----------
    X: np.array: Pre-action snapshot data. Columns are 
        timesteps, rows are features.
    Y: np.array: Post-action snapshot data. Columns are 
        timesteps, rows are features.
    rrr: bool: Whether or not to use Rayleigh-Ritz refinement
        Default is False which corresponds to not using RRR

    Outputs:
    -------
    eigvals: np.array: Eigenvalues of model
    eigvecs: np.array: Eigenvector array
    """
    U,s,Vh = np.linalg.svd(X,full_matrices=False)
    V = np.conjugate(np.transpose(Vh))
    Uh = np.conjugate(np.transpose(U))
    B = Y@V@np.diag(np.power(s,-1))
    eigvals,eigvecs = np.linalg.eig(Uh@B)
    if not(rrr):
        return eigvals,U@eigvecs
    if rrr:
        new_eigvecs = eigvecs.copy()
        for count,eig in enumerate(eigvals):
            M = B-eig*U
            _,_,Vh = np.linalg.svd(M)
            new_eigvecs[:,count] = np.conjugate(Vh[-1,:])
        return eigvals,new_eigvecs


In [ ]:
n_lift = 10 #Number of terms in the Lie Series to include
delays = 10 #Number of Delays to include 
ang_max=20 #Maximum Angular Velocity 
#Initialize lifted array, modified to haev correct shape
N = 24+9*n_lift
length = p_arr.shape[1]
lifted_arr = np.zeros((N,length))
#Initialize LCT to only form one time
LCT = hf.levi_cevita_tensor()
for j in range(length):
    lifted_arr[:,j] = form_lifting(p_arr[:,j],v_arr[:,j],
                                   R_arr[:,:,j],omega_r[:,j],n=n_lift,LCT=LCT,ang_max=ang_max)

delay_arr =hf.delay_embdedding(lifted_arr,d=delays)

#Seperate out X,Y and perform DMD
X = delay_arr[:,:-1]
Y = delay_arr[:,1:]
eigvals,eigvecs = basic_DMD(X,Y) 
p_delay_arr = p_arr[:,:delays]
v_delay = v_arr[:,:delays]
R_delay = R_arr[:,:,:delays]
omega_delay = omega_r[:,:delays]
init = form_lifting_delay(p_delay_arr,v_delay,R_delay,omega_delay,n=n_lift,LCT=LCT,ang_max=ang_max)
c0 = np.linalg.pinv(eigvecs)@init

In this eigenvalue, eigenvector pair, it is clear that we can propagate the steps via the Vandermonde matrix. Unfortunately, the default Vandermonde matrix is backwards for our requirements, so we must flip the matrix.

In [ ]:
prediction = np.flip(np.einsum("j,jk,ij -> ik",c0,np.vander(eigvals,n-delays+1),eigvecs),axis=1)

In [ ]:
base_size = 24+9*n_lift
x_prediction = prediction[-base_size,:]
y_prediction = prediction[-base_size+1,:]
z_prediction = prediction[-base_size+2,:]

Now let us add plot our predictions against the true values to test 

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
ax.set_title("Reconstruction using Lie Series and Time Delays")
x = p_arr[0,delays-1:]
y = p_arr[1,delays-1:]
z = p_arr[2,delays-1:]
ax.plot(x,y,z,label="True",color="orange")
ax.plot(x_prediction,y_prediction,z_prediction,label="Prediction",alpha=0.4,color="green")
ax.legend()
plt.show()
fig,axs = plt.subplots(1,3,tight_layout = True, figsize=(12,3))
axs[0].plot(x,label="True",color="orange")
axs[0].plot(x_prediction,label="Reconstruction",color="green",alpha=0.6)
axs[0].set_title("X Position")
axs[0].legend()
axs[1].plot(y,label="True",color="orange")
axs[1].plot(y_prediction,label="Reconstruction",color="green",alpha=0.6)
axs[1].legend()
axs[1].set_title("Y Position")
axs[2].plot(z,label="True",color="orange")
axs[2].plot(z_prediction,label="Reconstruction",color="green",alpha=0.6)
axs[2].legend()
axs[2].set_title("Z Position")
plt.show()

In [ ]:
pred = np.vstack((x_prediction,y_prediction,z_prediction))
true = np.vstack((x,y,z))
print(f"RMSE is {np.linalg.norm(pred-true)/np.sqrt(true.shape[1])}")

Now let us check the velocity reconstruction as well

In [ ]:
vx = prediction[-base_size+3,:]
vy = prediction[-base_size+4,:]
vz = prediction[-base_size+5,:]
fig,axs = plt.subplots(1,3,figsize=(9,3),tight_layout=True)
fig.suptitle("Tracking of Frame Velocity")
axs[0].plot(np.real(vx))
axs[0].set_title("X Velocity")
axs[1].plot(np.real(vy))
axs[1].set_title("Y Velocity")
axs[2].plot(np.real(vz))
axs[2].set_title("Z Velocity")
plt.show()

Note we do not enforce that our predction stay in SO(3), so we may check to determine if this is adequately maintained.  

In [ ]:
R_per = prediction[-base_size+6:-base_size+15,:].reshape((3,3,n-delays+1))
det_time_series = [np.real(np.linalg.det(R_per[:,:,k])) for k in range(n-delays+1)]
frobenius_time_series = [np.linalg.norm(R_per[:,:,k]@R_per[:,:,k].T-np.eye(3)) for k in range(n-delays+1)]
fig,axs = plt.subplots(1,2,figsize=(8,3),sharey=False,tight_layout=True)
fig.suptitle("Maintenance of SO(3)")
axs[0].plot(det_time_series)
axs[0].set_title("Determinant Time Series")
axs[1].plot(frobenius_time_series)
axs[1].set_title("Orthogonality Time Series")
plt.show()